# Задание №1

Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.

In [1]:
import pymongo
from pymongo import MongoClient

import requests
import json

from time import sleep
from pprint import pprint

import pandas as pd

Загружаем клиент

In [2]:
!brew services restart mongodb-community@6.0

Stopping `mongodb-community`... (might take a while)
==> Successfully stopped `mongodb-community` (label: homebrew.mxcl.mongodb-commu
==> Successfully started `mongodb-community` (label: homebrew.mxcl.mongodb-commu


In [3]:
client = MongoClient()

In [4]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

Создаем БД

In [5]:
db = client['HH_vacancies_db']

In [6]:
vacancy_info_collection = db['vacancy_info_collection']

Реализуем функцию, которая будет собирать информацию о вакансиях:

In [7]:
def hh_find_job_from_API(vacancy, page_number):
    url = 'https://api.hh.ru/vacancies'
    sleep(3)
    
    headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_2_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 OPR/95.0.4635.25',
    'content-encoding': 'gzip',
    'content-type': 'text/html'
        }
    params = {
        'text': f'NAME:{vacancy}', # Искомая вакансия
        'page': f'{page_number}', # Индекс страницы поиска на HH
        'per_page': 20 # Кол-во вакансий на 1 странице
    }
    
    response = requests.get('https://api.hh.ru/vacancies', params)
    data = response.content.decode() # Декодируем ответ, чтобы Кириллица отображалась корректно
    
    jsObj = json.loads(data)
    result = {}
    
    for i in jsObj['items']:
        name = i['name']
        result[name] = {
            'Salary': i['salary'],
            'Location': i['area']['name'],
            'Employer': i['employer']['name'],
            'Link': i['alternate_url']
        }

    with open('hh_find_job_from_API.json', 'w') as f:
        json.dump(result, f)
        
    hh_jobs = pd.read_json('hh_find_job_from_API.json').T
    return hh_jobs

In [8]:
hh_find_job_from_API('python', '0')

,Salary,Location,Employer,Link
Разработчик Python,"{'from': 100000, 'to': None, 'currency': 'RUR'...",Москва,Универсальная Электронная Платформа,https://hh.ru/vacancy/77234310
"Python-разработчик (Django, DRF)","{'from': 60000, 'to': 220000, 'currency': 'RUR...",Москва,Фабрика Решений,https://hh.ru/vacancy/70705466
Python программист (Django/Flask),"{'from': None, 'to': 180000, 'currency': 'RUR'...",Москва,Enjoypro,https://hh.ru/vacancy/77204132
Ведущий Python разработчик (удаленка),"{'from': None, 'to': 8000, 'currency': 'USD', ...",Москва,Apostro,https://hh.ru/vacancy/76832091
Программист Python - FastAPI,"{'from': 180000, 'to': None, 'currency': 'RUR'...",Астана,Digital агентство Piarme,https://hh.ru/vacancy/77244377
Программист Python,"{'from': 90000, 'to': None, 'currency': 'RUR',...",Санкт-Петербург,Парконика,https://hh.ru/vacancy/76331573
Python developer,None,Москва,АйТи Интеллект,https://hh.ru/vacancy/77302829
Python разработчик,"{'from': None, 'to': 200000, 'currency': 'RUR'...",Москва,Мамаева Ирина Анатольевна,https://hh.ru/vacancy/77032189
Программист Python Junior,None,Екатеринбург,BRANDPOL,https://hh.ru/vacancy/77146976
Python разработчик (аккредитованная компания),"{'from': 270000, 'to': None, 'currency': 'RUR'...",Москва,Ассоциация IPChain,https://hh.ru/vacancy/77071763


Реализуем функцию, которая будет коллекционировать данные в нашу бд:

In [9]:
def mongo_insert_data():
    with open('hh_find_job_from_API.json', 'r') as f:
        content = json.load(f)
    line = content.keys()
    result = []
    for line in content:
        data = {
            'Vacancy': line,
            'Salary': content.get(line)['Salary'],
            'Location':content.get(line)['Location'],
            'Employer':content.get(line)['Employer'],
            'Link': content.get(line)['Link']
        }
        result.append(data)
        
    counter = 0
    while counter < len(result):
        vacancy_info_collection.insert_one(result[counter])
        counter += 1
     
    return f'Data successfully inserted'

Для теста вызываем функцию несколько раз:

In [13]:
mongo_insert_data()

'Data successfully inserted'

In [14]:
vacancy_info_collection.count_documents({})

72

Избавимся от дубликатов:

In [15]:
def drop_duplicates(collection):
    df = pd.DataFrame(collection.find())
    df = df.drop_duplicates(subset=['Link'])
    df = df.to_dict('records')
    
    collection.delete_many({})
    collection.insert_many(df)
    return collection

In [16]:
drop_duplicates(vacancy_info_collection)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'HH_vacancies_db'), 'vacancy_info_collection')

In [17]:
vacancy_info_collection.count_documents({})

17

In [18]:
for i in enumerate(vacancy_info_collection.find()):
    pprint(i)

(0,
 {'Employer': 'Универсальная Электронная Платформа',
  'Link': 'https://hh.ru/vacancy/77234310',
  'Location': 'Москва',
  'Salary': {'currency': 'RUR', 'from': 100000, 'gross': False, 'to': None},
  'Vacancy': 'Разработчик Python',
  '_id': ObjectId('63f55706d0a5afb9036084bc')})
(1,
 {'Employer': 'Фабрика Решений',
  'Link': 'https://hh.ru/vacancy/70705466',
  'Location': 'Москва',
  'Salary': {'currency': 'RUR', 'from': 60000, 'gross': False, 'to': 220000},
  'Vacancy': 'Python-разработчик (Django, DRF)',
  '_id': ObjectId('63f55706d0a5afb9036084bd')})
(2,
 {'Employer': 'Apostro',
  'Link': 'https://hh.ru/vacancy/76832091',
  'Location': 'Москва',
  'Salary': {'currency': 'USD', 'from': None, 'gross': True, 'to': 8000},
  'Vacancy': 'Ведущий Python разработчик (удаленка)',
  '_id': ObjectId('63f55706d0a5afb9036084be')})
(3,
 {'Employer': 'Digital агентство Piarme',
  'Link': 'https://hh.ru/vacancy/77244377',
  'Location': 'Астана',
  'Salary': {'currency': 'RUR', 'from': 180000, 

# Задание №2

Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты).

In [34]:
def gt_salary(collection, salary, currency):
    result = []
    for i in collection.find():
        try:
            if i['Salary']['from'] > salary or i['Salary']['to'] > salary:
                if i['Salary']['currency'] == currency:
                    result.append(i)
        except TypeError:
            continue
            
    return result

In [36]:
gt_salary(vacancy_info_collection, 250000, 'RUR')

[{'_id': ObjectId('63f55706d0a5afb9036084c6'),
  'Vacancy': 'Python разработчик (аккредитованная компания)',
  'Salary': {'from': 270000, 'to': None, 'currency': 'RUR', 'gross': False},
  'Location': 'Москва',
  'Employer': 'Ассоциация IPChain',
  'Link': 'https://hh.ru/vacancy/77071763'},
 {'_id': ObjectId('63f55706d0a5afb9036084c7'),
  'Vacancy': 'Python Backend Developer / WEB - разработчик',
  'Salary': {'from': 300000, 'to': 450000, 'currency': 'RUR', 'gross': False},
  'Location': 'Москва',
  'Employer': 'Перфект Системс',
  'Link': 'https://hh.ru/vacancy/72216524'}]